In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [22]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    N = vec.shape[0]
    norm = np.linalg.norm(vec)     #комментировать особо нечего, реализую ту теорию сверху
    outvec = np.zeros(N)
    outvec[0] = norm
    
    
    u = (vec - outvec) / np.linalg.norm(vec - outvec)
    U = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            U[i, j] = u[i] * u[j]    #это я так u * u.T перемножаю, не нашел ф-ции в Numpy, *, @ или np.dot не подошли; P.S. потом узнал о существовании np.outer(), переписывать не стал 
    H = np.eye(N) - 2 * U
    
    return outvec, H
    # ... ENTER YOUR CODE HERE ...

Test your function using tests below:

In [23]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1, atol=1e-10) #Я изменил тут точность, он исходно стоит на 100% совпадение, хотя тест который перед ним не падает

In [24]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec) #вывода нет => переменные совпали

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [25]:
def qr_decomp(a):
    """
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    #a1 = np.array(a, copy=True, dtype=float) #я этим не вользовался, но оставил  как от исходного кода
    #m, n = a1.shape
    
    if a.shape[0] > a.shape[1]:    #в зависимости от того, какое измерение больше, зависит сколько циклов нужно до достижения верхнетреугольной
        N = a.shape[1]               #в общем N это кол-во диагоналей, элементы под которыми переходят в 0
    else: N = a.shape[0] - 1
    
    
    M = a.shape[0]         #M это именно кол-во строчек, оно используется как для определения размеров Q, так и для размеров матрицы "Хаусхолдера"
    r = a.copy()
    q = np.eye(M)
    
    for i in range(N):
        vec = r[i:, i]
        _, H = householder(vec)
        H_sq = np.eye(M)                #то есть чтобы добить нужный нам размер, все элементы сверху слева это единичная матрица
        H_sq[i:, i:] = H              #в следующей задаче я буду делать немного по-другому, каждый раз меняя только часть матрицы
        r = H_sq @ r               #сложность вычисления, как будет замечено ниже, m**2 * n, да еще и в цикле всё
        q = q @ H_sq.T
    
    return q, r
    # ... ENTER YOUR CODE HERE ...

In [26]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [27]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(3, 5))
print('Initial matrix: \n', a)
q, r = qr_decomp(a)
print('Decomposition: \n', 'Q: \n', q, '\n R: \n', r)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(3), atol=1e-10)    #Снова ничего не выводит => с некой точностью всё совпадает
                                                          #Проверил на разных матрицах, у которых или строчек, или столбцов больше, также на квадратных
# test the decomposition itself
assert_allclose(np.dot(q, r), a)

print('Q * R: \n', np.dot(q, r))

Initial matrix: 
 [[0.19151945 0.62210877 0.43772774 0.78535858 0.77997581]
 [0.27259261 0.27646426 0.80187218 0.95813935 0.87593263]
 [0.35781727 0.50099513 0.68346294 0.71270203 0.37025075]]
Decomposition: 
 Q: 
 [[ 0.39173836  0.89494087  0.21359281]
 [ 0.55756729 -0.41557276  0.71862229]
 [ 0.73188781 -0.16241956 -0.66178555]] 
 R: 
 [[ 0.48889634  0.76452352  1.11879064  1.36350018  1.06491985]
 [-0.          0.36048815 -0.05250354  0.18891613  0.27388252]
 [-0.          0.          0.21743282  0.38463134  0.55103534]]
Q * R: 
 [[0.19151945 0.62210877 0.43772774 0.78535858 0.77997581]
 [0.27259261 0.27646426 0.80187218 0.95813935 0.87593263]
 [0.35781727 0.50099513 0.68346294 0.71270203 0.37025075]]


Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [28]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)
print('True q: \n', qq, '\n True r: \n', rr)

True q: 
 [[-0.39173836  0.89494087 -0.21359281]
 [-0.55756729 -0.41557276 -0.71862229]
 [-0.73188781 -0.16241956  0.66178555]] 
 True r: 
 [[-0.48889634 -0.76452352 -1.11879064 -1.36350018 -1.06491985]
 [ 0.          0.36048815 -0.05250354  0.18891613  0.27388252]
 [ 0.          0.         -0.21743282 -0.38463134 -0.55103534]]


Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)

Можно заметить, что у Q различия в знаке по столбцам, у R - по строкам

(Можно увидеть, что по модулю все элементы матриц одинаковы):

In [9]:
assert_allclose(np.abs(q), np.abs(qq), atol=1e-10) #ничего не вывело => сошлось, но не всегда так, иногда элементы отличаются кардинально
assert_allclose(np.abs(r), np.abs(rr), atol=1e-10)

Думается, что это особенность вычисления qr в библиотеке scipy, в которой как-то хитро устроен алгоритм.

Причём документации я особо и не нашел, может не там искал, поэтому сказать что именно отличается не могу.

Единственная догадка - допустим есть особая матрица А, при применении которой к верхнетреугольной она ее не меняет; тогда пусть у нее также есть обратная; при вставке между Q и R матрицы A^-1 * A, по1меняться ничего не должно, однако Q и R, теперь другие, всё равно дающие исходную матрицу => неоднозначность QR-decomp? Однако до какого-то конкретного вида A я не додумал

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [19]:
#всё, что нужно поменять - вывод 1 функции, сделать householder_mod(), которая возвращает u вместо H
def householder_mod(vec):
    """
    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    u : array of floats, shape(n,)
        Reflection vector for the given x and 1st coordinate
    
    """
    
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    N = vec.shape[0]
    norm = np.linalg.norm(vec)     #урезанная функция, которая уже не считает u * u.T
    outvec = np.zeros(N)
    outvec[0] = norm
    
    u = (vec - outvec) / np.linalg.norm(vec - outvec)
    
    return u

#теперь немного преображенная qr_decomp_matrix_R(), возвращает только часть разложения, а именно матрицу R, далее отдельно будет функция для Q

def qr_decomp_matrix_R(a):
    """
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    r : ndarray, shape(m, n)
        The upper triangular matrix
    U : ndarray, shape(m, m)
        The array of reflection vectors u of sub-diagonal vectors of a matrix
    
    """
    if a.shape[0] > a.shape[1]:
        N = a.shape[1]
    else: N = a.shape[0] - 1
    
    
    M = a.shape[0]    #алгоритм такой же, единственно что теперь придётся помучаться с перемножением u на матрицу слева
    r = a.copy()
    q = np.eye(M)
    U = np.ones((N,M))       #тут будут по строчкам храниться вектора, но т.к. они разной размерности то перед вектором будет стоять посл-ть из 1(можно было бы сделать из любого числа)
    for i in range(N):
        vec = r[i:, i]
        u = householder_mod(vec)
        r_slice = r[i:, i:]                  #снова возникает проблема с умножением строки на матрицу и столбца на строку
        
        Temp1 = np.zeros(r_slice.shape[1])     #в этих матрицах будет находиться это сердобольное перемножение
        Temp2 = np.zeros_like(r_slice)
        for k in range(Temp1.shape[0]):
                Temp1[k] = u @ r_slice[:, k]     #это часть u.T @ A
        
        for k in range(Temp2.shape[0]):
            for j in range(Temp2.shape[1]):
                Temp2[k, j] = u[k] * Temp1[j]   #это часть u @ (u.T @ A)
        
        r_slice = r_slice - 2 * Temp2       #Это была расписана матрица Хаусхолдера
        
        r[i:, i:] = r_slice       #добавляем наш вектор нормали, и заменяем нижнюю правую часть матрицы новой; цикл заново
        U[i, i:] = u
    
    
    return r, U

#Теперь пробуем восстановить Q по нашим векторам из U, это должно быть так же не трудно, как и R
def qr_decomp_matrix_Q(a, U):
    """
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    U : ndarray, shape(m, m)
        The array of reflection vectors u of sub-diagonal vectors of a matrix
    Returns
    -------
    Q_t : 
        ndarray, shape(m, n)
        Transposed matrix q of the qr decomposition
    """
    N = U.shape[0]            #Мы знаем, сколько именно векторов, по построению U
    
    Q_t = np.eye(a.shape[0])        #Тут я тоже H не строил явно
    
    for i in range(N):
        u = U[i, i:]                       # "Извлекаю" вектор, так просто будет понятнее что происходит;
        
        Temp1 = np.zeros(a.shape[0])
        Temp2 = np.zeros((u.shape[0], Temp1.shape[0]))
        
        for k in range(Temp1.shape[0]):            #если расписать  по-честному перемножение матрицы H_sq(эта матрица уже использовалась), то
            Temp1[k] = u @ Q_t[i:, k]                #получится что u.T * u действует только на нижние строки следующей матрицы, что собственно здесь и написано
        
        for k in range(Temp2.shape[0]):
            for j in range(Temp2.shape[1]):
                Temp2[k, j] = u[k] * Temp1[j]           #вместо a у нас изначально есть матрица единичная, и тогда мы получаем H_k * H_k-1 * ... * H_1 * (A * A**-1) = R * A**-1 = Q_t
                                                    #тогда на каждом шаге мы урезаем часть, на которую действует наш u.T * u, в соответствии с длиной этого самого u
        Q_t[i:, :] -= 2 * Temp2
                                                #я вывожу именно Q_t, т.к. для меня такой вывод функции несет больший смысл, всё-таки мы
    return Q_t                              #получаем обе части qr-разложения, а проверить результат можно уже вне функции


In [20]:
r1, U1 = qr_decomp_matrix_R(a)
print('Initial matrix: \n', a, '\n')

print('R part of QR decomposition: \n', r1,'\n')

Q_t = qr_decomp_matrix_Q(a, U1)
print('Q_t part of QR decomposition: \n', Q_t, '\n')
R1 = Q_t @ a
print('Check if Q_t * A equals R: \n', R1)
assert_allclose(r1, R1, atol=1e-10)
print('Ok')

Initial matrix: 
 [[0.19151945 0.62210877 0.43772774 0.78535858 0.77997581]
 [0.27259261 0.27646426 0.80187218 0.95813935 0.87593263]
 [0.35781727 0.50099513 0.68346294 0.71270203 0.37025075]] 

R part of QR decomposition: 
 [[ 0.48889634  0.76452352  1.11879064  1.36350018  1.06491985]
 [-0.          0.36048815 -0.05250354  0.18891613  0.27388252]
 [-0.          0.          0.21743282  0.38463134  0.55103534]] 

Q_t part of QR decomposition: 
 [[ 0.39173836  0.55756729  0.73188781]
 [ 0.89494087 -0.41557276 -0.16241956]
 [ 0.21359281  0.71862229 -0.66178555]] 

Check if Q_t * A equals R: 
 [[ 0.48889634  0.76452352  1.11879064  1.36350018  1.06491985]
 [ 0.          0.36048815 -0.05250354  0.18891613  0.27388252]
 [-0.         -0.          0.21743282  0.38463134  0.55103534]]
Ok


Ну а теперь сделаем несколько тестов на матрицах, с разными формами/кол-вом элементов:

In [29]:
Seeds = np.array([np.random.RandomState(i**2 + 4*i +7) for i in range(5)])
Test0 = Seeds[0].uniform(size=(6,6))  #Квадратная матрица
Test1 = Seeds[1].uniform(size=(3,7))  #Прямоугольная, столбцов больше
Test2 = Seeds[2].uniform(size=(8,4))  #Прямоугольная, строк больше
Test3 = Seeds[3].uniform(size=(250, 300))  #Просто большая, ее выводить не будем, сравним с помощью assert_allclose()

R_0, U_0 = qr_decomp_matrix_R(Test0)
Q_t_0 = qr_decomp_matrix_Q(Test0, U_0)

print('Initial matrix: \n', Test0, '\n')
print('Q @ R: \n', Q_t_0.T @ R_0, '\n\n\n')
assert_allclose(Test0, Q_t_0.T @ R_0, atol=1e-10)

#------------------------------------------------------------

R_1, U_1 = qr_decomp_matrix_R(Test1)
Q_t_1 = qr_decomp_matrix_Q(Test1, U_1)

print('Initial matrix: \n', Test1, '\n')
print('Q @ R: \n', Q_t_1.T @ R_1, '\n\n\n')
assert_allclose(Test1, Q_t_1.T @ R_1, atol=1e-10)

#------------------------------------------------------------

R_2, U_2 = qr_decomp_matrix_R(Test2)
Q_t_2 = qr_decomp_matrix_Q(Test2, U_2)

print('Initial matrix: \n', Test2, '\n')
print('Q @ R: \n', Q_t_2.T @ R_2, '\n\n\n')
assert_allclose(Test2, Q_t_2.T @ R_2, atol=1e-10)

#------------------------------------------------------------

R_3, U_3 = qr_decomp_matrix_R(Test3)
Q_t_3 = qr_decomp_matrix_Q(Test3, U_3)

print('Check for big matrix: ')
assert_allclose(Test3, Q_t_3.T @ R_3, atol=1e-10)
print('Ok')  #Т.к. эта функция ничего не возвращает, но если все ок то пропускает код дальше, можно написать именно так, print после неё

Initial matrix: 
 [[0.07630829 0.77991879 0.43840923 0.72346518 0.97798951 0.53849587]
 [0.50112046 0.07205113 0.26843898 0.4998825  0.67923    0.80373904]
 [0.38094113 0.06593635 0.2881456  0.90959353 0.21338535 0.45212396]
 [0.93120602 0.02489923 0.60054892 0.9501295  0.23030288 0.54848992]
 [0.90912837 0.13316945 0.52341258 0.75040986 0.66901324 0.46775286]
 [0.20484909 0.49076589 0.37238469 0.47740115 0.36589039 0.83791799]] 

Q @ R: 
 [[0.07630829 0.77991879 0.43840923 0.72346518 0.97798951 0.53849587]
 [0.50112046 0.07205113 0.26843898 0.4998825  0.67923    0.80373904]
 [0.38094113 0.06593635 0.2881456  0.90959353 0.21338535 0.45212396]
 [0.93120602 0.02489923 0.60054892 0.9501295  0.23030288 0.54848992]
 [0.90912837 0.13316945 0.52341258 0.75040986 0.66901324 0.46775286]
 [0.20484909 0.49076589 0.37238469 0.47740115 0.36589039 0.83791799]] 



Initial matrix: 
 [[0.15416284 0.7400497  0.26331502 0.53373939 0.01457496 0.91874701
  0.90071485]
 [0.03342143 0.95694934 0.13720932 0.